In [72]:
import glob, os
from itertools import product
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn import preprocessing

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.neighbors import LocalOutlierFactor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

import collections 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold, PredefinedSplit
from sklearn.metrics import accuracy_score
import warnings 


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from pytimekr import pytimekr
warnings.filterwarnings(action="ignore")
%matplotlib inline

# Load the dataset
data = pd.read_csv('./train.csv')
data.head()

plt.rc("font", family="AppleGothic")
plt.rcParams['axes.unicode_minus'] = False


In [73]:
data_test = pd.read_csv('./test.csv')

In [74]:
data['item'].value_counts()

item
TG    15230
BC    13707
RD    12184
CR    10661
CB     7615
Name: count, dtype: int64

In [75]:
data.isna().sum()

ID             0
timestamp      0
item           0
corporation    0
location       0
supply(kg)     0
price(원/kg)    0
dtype: int64

In [76]:
type(data['timestamp'][0])

str

In [77]:
import datetime

data['timestamp'] = pd.to_datetime(data['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data['year'] = data['timestamp'].dt.isocalendar().year
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['week'] = data['timestamp'].dt.isocalendar().week

data['weekday'] = data['timestamp'].dt.isocalendar().day



In [78]:
import datetime

data_test['timestamp'] = pd.to_datetime(data_test['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data_test['year'] = data_test['timestamp'].dt.isocalendar().year
data_test['month'] = data_test['timestamp'].dt.month
data_test['day'] = data_test['timestamp'].dt.day
data_test['week'] = data_test['timestamp'].dt.isocalendar().week

data_test['weekday'] = data_test['timestamp'].dt.isocalendar().day



In [79]:
data['isWeekday'] = ((data['weekday'] >= 1) & (data['weekday'] <= 5)).astype(int)
data['isSaturday'] = (data['weekday'] == 6).astype(int)
data['isSunday'] = (data['weekday'] == 7).astype(int)

In [80]:
data_test['isWeekday'] = ((data_test['weekday'] >= 1) & (data_test['weekday'] <= 5)).astype(int)
data_test['isSaturday'] = (data_test['weekday'] == 6).astype(int)
data_test['isSunday'] = (data_test['weekday'] == 7).astype(int)

In [81]:
# weekday 삭제 
data.drop('weekday', axis=1,inplace=True)


In [82]:
data_test.drop('weekday', axis=1,inplace=True)

In [83]:
data

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,week,isWeekday,isSaturday,isSunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,1,1,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,1,1,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,1,1,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,9,1,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,9,1,0,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,9,1,0,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,9,1,0,0


In [84]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [92]:
# Encoding categorical variables and scaling
categorical_features = ['item', 'corporation', 'location', 'year','isWeekday','isSaturday','isSunday']
numeric_features = ['month', 'day', 'week']

# Pipeline for transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [93]:
data_test[numeric_features[:-1] + categorical_features]

,month,day,item,corporation,location,year,isWeekday,isSaturday,isSunday
0,3,4,TG,A,J,2023,0,1,0
1,3,5,TG,A,J,2023,0,0,1
2,3,6,TG,A,J,2023,1,0,0
3,3,7,TG,A,J,2023,1,0,0
4,3,8,TG,A,J,2023,1,0,0
...,...,...,...,...,...,...,...,...,...
1087,3,27,RD,F,J,2023,1,0,0
1088,3,28,RD,F,J,2023,1,0,0
1089,3,29,RD,F,J,2023,1,0,0
1090,3,30,RD,F,J,2023,1,0,0


In [94]:
# Apply transformations to training data
train_data_preprocessed = preprocessor.fit_transform(data[numeric_features + categorical_features])

# Apply the same transformations to test data
# Note: We do not have 'supply(kg)' in test data, so it's excluded
test_data_preprocessed = preprocessor.transform(data_test[numeric_features[:-1]+ categorical_features])

# Display the shape of the preprocessed data as a check
train_data_preprocessed.shape, test_data_preprocessed.shape

KeyError: "['week'] not in index"

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import mean_squared_error


# Assumed number of timesteps for the LSTM model
timesteps = 10

# Reshape the training data to fit the LSTM model
# LSTM requires the input shape of the form [samples, timesteps, features]

# Function to create a dataset suitable for LSTM input
def create_lstm_dataset(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:(i + n_steps), :])
        y.append(data[i + n_steps, -1])
    return np.array(X), np.array(y)


# Creating the LSTM datasets
X_lstm, y_lstm = create_lstm_dataset(np.hstack((train_data_preprocessed, y_train.reshape(-1, 1))), timesteps)
X_lstm.shape, y_lstm.shape

# Setting up the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(timesteps, X_lstm.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Training the model
model.fit(X_lstm, y_lstm, epochs=10, batch_size=32, verbose=1)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE: ", rmse)


AttributeError: 'Series' object has no attribute 'reshape'